In [2]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import nlp
import random
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [3]:
dataset = nlp.load_dataset('emotion')
train = dataset['train']
val = dataset['validation']
test = dataset['test']

Downloading:   0%|          | 0.00/3.41k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/6.01k [00:00<?, ?B/s]

Using custom data configuration default


Downloading:   0%|          | 0.00/1.66M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/204k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/207k [00:00<?, ?B/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset emotion downloaded and prepared to /root/.cache/huggingface/datasets/emotion/default/0.0.0/84e07cd366f4451464584cdbd4958f512bcaddb1e921341e07298ce8a9ce42f4. Subsequent calls will reuse this data.


In [4]:
def get_tweet(data):
    tweets = [x['text'] for x in data]
    labels = [x['label'] for x in data]
    return tweets, labels
tweets, labels = get_tweet(train)

In [5]:
tweets[0], labels[0]

('i didnt feel humiliated', 'sadness')

In [6]:
tokenizer = Tokenizer(num_words=10000, oov_token='<UNK>')
tokenizer.fit_on_texts(tweets)

In [7]:
maxlen=50
def get_sequences(tokenizer, tweets):
    sequences = tokenizer.texts_to_sequences(tweets)
    padded = pad_sequences(sequences, truncating = 'post', padding='post', maxlen=maxlen)
    return padded

In [8]:
classes = set(labels)
class_to_index = dict((c,i) for i, c in enumerate(classes))
index_to_class = dict((v,k) for k, v in class_to_index.items())
names_to_ids = lambda labels: np.array([class_to_index.get(x) for x in labels])
train_labels = names_to_ids(labels)

In [9]:
model = tf.keras.models.Sequential([
tf.keras.layers.Embedding(10000,16,input_length=maxlen),
tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(20, return_sequences=True)),
tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(20)),
tf.keras.layers.Dense(6, activation='softmax')
])


In [10]:
model.compile(
     loss='sparse_categorical_crossentropy',
     optimizer='adam',
     metrics=['accuracy']
)

In [12]:
padded_train_seq = get_sequences(tokenizer, tweets)

In [13]:
val_tweets, val_labels = get_tweet(val)
val_seq = get_sequences(tokenizer, val_tweets)
val_labels= names_to_ids(val_labels)
h = model.fit(
     padded_train_seq, train_labels,
     validation_data=(val_seq, val_labels),
     epochs=20,
     callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=2)]
)

Epoch 1/20
500/500 [==============================] - 22s 34ms/step - loss: 1.3228 - accuracy: 0.4622 - val_loss: 0.8260 - val_accuracy: 0.7035
Epoch 2/20
500/500 [==============================] - 15s 30ms/step - loss: 0.5975 - accuracy: 0.7966 - val_loss: 0.6653 - val_accuracy: 0.7865
Epoch 3/20
500/500 [==============================] - 15s 30ms/step - loss: 0.4125 - accuracy: 0.8633 - val_loss: 0.5309 - val_accuracy: 0.8190
Epoch 4/20
500/500 [==============================] - 15s 30ms/step - loss: 0.2986 - accuracy: 0.8972 - val_loss: 0.5092 - val_accuracy: 0.8435
Epoch 5/20
500/500 [==============================] - 15s 30ms/step - loss: 0.2456 - accuracy: 0.9166 - val_loss: 0.5126 - val_accuracy: 0.8330
Epoch 6/20
500/500 [==============================] - 15s 30ms/step - loss: 0.2162 - accuracy: 0.9239 - val_loss: 0.4799 - val_accuracy: 0.8590
Epoch 7/20
500/500 [==============================] - 15s 30ms/step - loss: 0.1930 - accuracy: 0.9394 - val_loss: 0.5592 - val_accuracy:

In [14]:
test_tweets, test_labels=get_tweet(test)
test_seq = get_sequences(tokenizer, test_tweets)
test_labels=names_to_ids(test_labels)
model.evaluate(test_seq, test_labels)

63/63 [==============================] - 1s 10ms/step - loss: 0.4754 - accuracy: 0.8765


[0.4753565192222595, 0.8765000104904175]

In [15]:
i = random.randint(0,len(test_labels)-1)
print('Sentence:', test_tweets[i])
print('Emotion:', index_to_class[test_labels[i]])
p = model.predict(np.expand_dims(test_seq[i], axis=0))[0]
print(test_seq[i])
pred_class=index_to_class[np.argmax(p).astype('uint8')]
print('Predicted Emotion: ', pred_class)

Sentence: i also tell you in hopes that anyone who is still feeling stigmatized or ashamed of their mental health issues will let go of the stigma let go of the shame
Emotion: sadness
[   2  117  221   31   12 2047    9  244   76   22   72    8    1   35
  406   10  111 1378 1030 1000   59  207  101   10    6    1  207  101
   10    6 1875    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0]
Predicted Emotion:  sadness


In [17]:
sentence = 'i am going to suicide'
sequence = tokenizer.texts_to_sequences([sentence])
paddedSequence = pad_sequences(sequence, truncating = 'post', padding='post', maxlen=maxlen)
p = model.predict(np.expand_dims(paddedSequence[0], axis=0))[0]
pred_class=index_to_class[np.argmax(p).astype('uint8')]
print('Sentence:', sentence)
print('Predicted Emotion: ', pred_class)

Sentence: i am going to suicide
Predicted Emotion:  sadness
